In [1]:
suppressMessages(library(rwwa))

# Fit linear model to gridded data

In [2]:
ds_nm = "mswep"

# load covariate data
gmst = read.table("gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2025]

# define factual & counterfactual covariates
cov_f <- gmst[gmst$year == 2025,"gmst", drop = F]
cov_cf <- rbind("pi" = cov_f - 1.3)

In [3]:
# loop over all flattened data & fit nonstationary model to each - NB no bootstrapping
fl = list.files("flattened", pattern = ds_nm, full.names = T)
for(fnm in fl) {
    
    res_fnm <- gsub("flattened", "flattened-res", fnm)
    
    if (!file.exists(res_fnm)) {
        
        # load the .csv
        df <- read.csv(fnm, skip = 3, header = F)
        colnames(df)[1] <- "year"
        ev <- df[df$year == 2026,]
        df <- merge(gmst, df)

        # fit nonstationary model to all columns
        models <- sapply(colnames(df)[grepl("V", colnames(df))], function(cnm) {
            df_c <- df[,c("year", "gmst", cnm)]
            ev_c <- ev[,cnm]
            tryCatch({
                fit_ns("gev", "fixeddisp", df_c, cnm, c("gmst"), lower = F, ev = ev_c)
            }, error = function(cond) { return(NULL) })
        }, simplify = F)

        # get all model results & save as .csv to be reconstructed & plotted in python
        fitted_res <- sapply(models, function(mdl) {
            if(!is.null(unlist(mdl)[1])) {
                mdl_ests(mdl, cov_f = cov_f, cov_cf = cov_cf)
            } else {
                rep(NA, 10)
            }})
        write.csv(fitted_res, res_fnm)
    }
}

# Fit linear model to station data

In [49]:
# load covariate data
gmst = read.table("gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2025]

# define factual & counterfactual covariates
cov_f <- gmst[gmst$year == 2025,"gmst", drop = F]
cov_cf <- rbind("pi" = cov_f - 1.3)

In [54]:
df <- read.csv("stations_rx7day.csv")
df <- merge(gmst, df)

# fit nonstationary model to all columns
models <- sapply(colnames(df)[grepl("X", colnames(df))], function(cnm) {
    df_c <- df[,c("year", "gmst", cnm)]
    df_c <- df_c[!is.na(df_c[,cnm]),]
    tryCatch({
        fit_ns("gev", "fixeddisp", df_c, cnm, c("gmst"), lower = F)
    }, error = function(cond) { return(NULL) })
}, simplify = F)

# get all model results & save as .csv to be reconstructed & plotted in python
fitted_res <- sapply(models, function(mdl) {
    if(!is.null(unlist(mdl)[1])) {
        c(mdl_ests(mdl, cov_f = cov_f, cov_cf = cov_cf), "nobs" = length(mdl$x))
    } else {
        c(rep(NA, 11))
    }})

write.csv(fitted_res, "fitted-res_stations-rx7day.csv")

## Station average